作业第二周 MLP模型练习fashion_mnist分类操作

仿照课件完成fashion_mnist的分类操作：
1.练习keras内置数据集fashion_mnist的读取与操作。
Fashion-MNIST克隆了MNIST的所有外在特征： 60000张训练图像和对应Label； 10000张测试图像和对应Label； 10个类别；
(train_images, train_labels),(test_images,test_labels)= tf.keras.datasets.fashion_mnist.load_data()

2.设计一个简单多层感知机网络，训练fashion_mnist的分类操作。
(打印loss变化曲线曲线，显示测试集最后的预测准确率、混淆矩阵、典型误判图像等)

In [1]:
from tensorflow import keras
from keras.utils import np_utils
import numpy as np
import os

from keras.datasets import fashion_mnist
(train_images, train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [ ]:
#train_labels.min(),train_labels.max()
#train_images[0], train_labels[0]

In [4]:
x_train = train_images.reshape(60000, 784).astype('float32')
x_test = test_images.reshape(10000, 784).astype('float32')
# x_train = train_images.astype('float32')
# x_test = test_images.astype('float32')
x_train_norm = x_train / 255
x_test_norm = x_test / 255
y_train_onehot = np_utils.to_categorical(train_labels)
y_test_onehot = np_utils.to_categorical(test_labels)

In [5]:
print(x_train_norm.shape, y_train_onehot.shape)

(60000, 28, 28) (60000, 10)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(
    units=256, input_dim = 784,
    kernel_initializer='normal',
    activation='relu'
))
model.add(Dense(
    units=10, kernel_initializer='normal',
    activation='softmax'
))

In [ ]:
print(model.summary())

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x=x_train_norm, y=y_train_onehot,
                    validation_split=0.2, epochs=20,
                    batch_size=200, verbose=2)

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(history, train, validation):
    plt.plot(history.history[train])
    plt.plot(history.history[validation])
    plt.title('train history')
    plt.ylabel(train)
    plt.xlabel('epoch')
    plt.legend(['train','validation'], loc='upper left')
    plt.grid()
    plt.show()

show_train_history(history,'accuracy','val_accuracy')
show_train_history(history,'loss','val_loss')

In [ ]:
scores = model.evaluate(x_test_norm, y_test_onehot)
print()
print('accuracy=', scores[1])
print(scores)

In [ ]:
prediction = np.argmax(model.predict(x_test_norm), axis=-1)
prediction

In [ ]:
mis_predicted = []
for i in range(prediction.shape[0]):
    if test_labels[i] != prediction[i]:
        mis_predicted.append(i)
print(mis_predicted)

In [ ]:
def plot_labels_prediction(images, labels, prediction, idx, num=10):
    fig = plt.gcf()
    fig.set_size_inches(12,14)
    if num>25: num=25
    for i in range(0, num):
        ax = plt.subplot(5, 5, i+1)
        ax.imshow(images[idx], cmap='binary')
        title = 'label=' + str(labels[idx])
        if len(prediction)>0:
            title += ',predit=' + str(prediction[idx])
        
        ax.set_title(title, fontsize=10)
        ax.set_xticks([]);ax.set_yticks([])
        idx += 1
    plt.show()

plot_labels_prediction(test_images, test_labels, prediction, idx=110)

In [ ]:
import pandas as pd
pd.crosstab(test_labels, prediction, rownames=['label'], colnames=['predict'])

In [ ]:
plot_labels_prediction(test_images, test_labels, prediction, idx=1665, num=1)